## 初期セットアップ

In [ ]:
!pip install -r requirements.txt

In [ ]:
import uuid

# セッションIDの生成
session_id = str(uuid.uuid4())

In [ ]:
from bedrock_agentcore.services.identity import IdentityClient
from boto3.session import Session

import os

boto_session = Session()
region = boto_session.region_name

# Identity Client の初期化
identity_client = IdentityClient(region=region)

# Entra ID の設定 (ユーザー認証用)
os.environ["entra_client_id"] = "xxxxx"
os.environ["entra_scopes"] = "api://xxxxx/qiita_test"
os.environ["entra_tenant_id"] = "xxxxx"
os.environ["entra_audience"] = "api://xxxxx"

### msal を使った Entra ID からのアクセストークン取得テスト

In [ ]:
import msal
import webbrowser

REDIRECT_URI = (
    f"https://bedrock-agentcore.{region}.amazonaws.com/identities/oauth2/callback"
)
AUTHORITY = f"https://login.microsoftonline.com/{os.environ['entra_tenant_id']}"

app = msal.PublicClientApplication(
    client_id=os.environ["entra_client_id"],
    authority=AUTHORITY,
)

entra_auth_result = app.acquire_token_silent(
    scopes=[os.environ["entra_scopes"]], account=None
)

if not entra_auth_result:
    flow = app.initiate_device_flow(scopes=[os.environ["entra_scopes"]])
    if "user_code" not in flow:
        raise ValueError("デバイスフローの開始に失敗しました。")

    print(flow["message"])
    webbrowser.open(flow["verification_uri"])

    entra_auth_result = app.acquire_token_by_device_flow(flow)

access_token = entra_auth_result["access_token"]

print(f"Bearerトークンの取得完了: {access_token[:20]}...")

## Atlassian Provider のデプロイ

In [ ]:
from bedrock_agentcore.services.identity import IdentityClient
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name

identity_client = IdentityClient(region=region)

# Atlassian OAuth 2.0 アプリケーションの設定
os.environ["atlassian_client_id"] = "xxxxx"
os.environ["atlassian_secret"] = "xxxxx"
os.environ["atlassian_scopes"] = (
    "read:page:confluence write:page:confluence read:space:confluence read:space-details:confluence search:confluence offline_access"
)

# Provider（アイデンティティ）の作成
atlassian_provider = identity_client.create_oauth2_credential_provider(
    req={
        "name": "atlassian_oauth_provider",
        "credentialProviderVendor": "AtlassianOauth2",
        "oauth2ProviderConfigInput": {
            "atlassianOauth2ProviderConfig": {
                "clientId": os.environ["atlassian_client_id"],
                "clientSecret": os.environ["atlassian_secret"],
            }
        },
    }
)

## Microsoft Entra を使ったインバウンド認証のテスト

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime

agentcore_runtime = Runtime()

discovery_url = f"https://login.microsoftonline.com/{os.environ['entra_tenant_id']}/.well-known/openid-configuration"

entra_runtime_response = agentcore_runtime.configure(
    entrypoint="strands_with_memory_sample.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_entra_inbound",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,
            "allowedAudience": [os.environ["entra_audience"]],
        }
    },
)

In [ ]:
strands_entraid_response = agentcore_runtime.launch(local_build=True)

In [ ]:
# メモリの状態が'ACTIVE'になるまで待機するユーティリティ関数
import time


def wait_for_memory_active(memory_id: str, control_client, max_wait_seconds: int = 300):
    """メモリの状態が'ACTIVE'になるまで5秒間隔でポーリング"""
    start_time = time.time()

    while True:
        response = control_client.get_memory(memoryId=memory_id)
        status = response["memory"]["status"]

        if status == "ACTIVE":
            return response
        elif status == "FAILED":
            raise ValueError(f"Memory failed: {memory_id}")
        elif status == "DELETING":
            raise ValueError(f"Memory is being deleted: {memory_id}")

        elapsed = int(time.time() - start_time)
        if elapsed >= max_wait_seconds:
            raise TimeoutError(
                f"Timeout waiting for memory to become active (status: {status})"
            )

        print(f"Waiting for memory... (status: {status}, {elapsed}s elapsed)")
        time.sleep(5)

In [ ]:
import boto3

# boto3クライアントの初期化
agentcore_control_client = boto3.client("bedrock-agentcore-control", region_name=region)

# メモリIDの取得
memory_id_1 = agentcore_control_client.get_agent_runtime(
    agentRuntimeId=strands_entraid_response.agent_id
)["environmentVariables"]["BEDROCK_AGENTCORE_MEMORY_ID"]

# メモリがActiveになるまで待機
wait_for_memory_active(memory_id_1, agentcore_control_client)

# invoke実行
invoke_response = agentcore_runtime.invoke(
    payload={
        "prompt": "こんにちはーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー！！！！！！！！！！！！！！！！！！！！！！！！！！！！！",
    },
    bearer_token=access_token,
    session_id=session_id,
    user_id="user1",
)

invoke_response

### AgentCore.Identity ライブラリを使った Atlassian 認証テスト

In [ ]:
# プロバイダを使用してアクセストークンを取得
from bedrock_agentcore.identity.auth import requires_access_token


@requires_access_token(
    provider_name="atlassian_oauth_provider",
    auth_flow="USER_FEDERATION",
    scopes=os.environ["atlassian_scopes"].split(" "),
    on_auth_url=lambda x: print(
        "\n表示されたURLをブラウザのアドレスバーにコピーし認証を完了させてください:\n"
        + x
    ),
    force_authentication=True,
)
def need_atlassian_access_token(*, access_token: str):
    return access_token


need_atlassian_access_token()

## Runtime のデプロイ

In [ ]:
agentcore_runtime = Runtime()

strands_e_a_response = agentcore_runtime.configure(
    entrypoint="strands_confluence.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_entra_confluence_3lo",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,
            "allowedAudience": [os.environ["entra_audience"]],
        }  # ここでインバウンド認証の設定を追加
    },
)

In [ ]:
launch_response = agentcore_runtime.launch(
    local_build=True,
    auto_update_on_conflict=True,
    env_vars={
        "atlassian_scopes": os.environ["atlassian_scopes"],
    },
)

## Runtime にデプロイされたエージェントの実行 

### Confluence ページの検索と取得

In [ ]:
# Entra IDからユーザーIDを取得（アクセストークン指定時はトークンの情報でユーザーが識別される）
# agentcore_user_id = entra_auth_result["id_token_claims"]["oid"]

# メモリIDの取得
memory_id_2 = agentcore_control_client.get_agent_runtime(
    agentRuntimeId=launch_response.agent_id
)["environmentVariables"]["BEDROCK_AGENTCORE_MEMORY_ID"]

# メモリがActiveになるまで待機
wait_for_memory_active(memory_id_2, agentcore_control_client)

prompt = "鳥貴族に関するページをConfluenceで検索し、内容を要約してください。"

result = agentcore_runtime.invoke(
    payload={"prompt": prompt},
    session_id=session_id,
    # user_id=agentcore_user_id,
    bearer_token=access_token,
)

### Confluence ページの新規作成

In [ ]:
prompt = "「わっしょいスペース」というスペースに、Amazon Bedrockに関する解説ページを新規作成してください。内容は200字程度でお願いします。"

result = agentcore_runtime.invoke(
    payload={"prompt": prompt},
    session_id=session_id,
    # user_id=agentcore_user_id,
    bearer_token=access_token,
)

### 無効なアクセストークンを指定した場合

In [ ]:
prompt = "「わっしょいスペース」というスペース（ID：wasshoi）に、わっしょい祭り2025に関する解説ページを新規作成してください。内容は200字程度でお願いします。"

result = agentcore_runtime.invoke(
    payload={"prompt": prompt},
    session_id=session_id,
    # user_id=agentcore_user_id,
    bearer_token="wasshoi_festival_2025",  # 無効なトークンを指定
)

## AgentCore リソース削除

In [ ]:
import boto3

agentcore_control_client = boto3.client("bedrock-agentcore-control", region_name=region)

# メモリの削除
agentcore_control_client.delete_memory(memoryId=memory_id_1)
agentcore_control_client.delete_memory(memoryId=memory_id_2)

# ランタイムの削除
agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=strands_entraid_response.agent_id
)
agentcore_control_client.delete_agent_runtime(agentRuntimeId=launch_response.agent_id)

# プロバイダーの削除
agentcore_control_client.delete_oauth2_credential_provider(
    name=atlassian_provider["name"]
)